In [16]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_csv('bronze/Properties.csv', encoding='utf-8', sep=';')
df

,property_url,broker,district,price,type,city,state,ref,category,area,lat,long,num_bedroom,num_bathroom,num_garage,added_date,sold_date
0,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Zona 08,Consulte-nos,Padrão,Maringa,PR,5363.0,Apartamento,"55,34 m²",-23.446941,-51.919694,2,1,-,2025-04-15,NaN
1,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Zona 01,"R$ 90.000,00",Padrão,Maringá,PR,31223.0,Apartamento,"48,40 m²",-23.421000,-51.933056,1,1,-,2025-04-15,NaN
2,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Zona 01,"R$ 126.312,10",Padrão,Maringá,PR,35423.0,Apartamento,"66,36 m²",-23.421000,-51.933056,1,1,-,2025-04-15,NaN
3,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Jardim Barcelona,"R$ 135.000,00",Padrão,Maringá,PR,82763.0,Terreno,"300,00 m²",-23.459239,-51.981950,-,-,-,2025-04-15,NaN
4,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Jardim Novo Paulista,"R$ 140.000,00",Padrão,Maringá,PR,48803.0,Terreno,"200,00 m²",-23.408781,-51.865269,-,-,-,2025-04-15,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482,https://www.silvioiwata.com.br/imovel/36200064...,Silvio Iwata,Zona 03,"R$ 570.000,00",NaN,Maringá,PR,NaN,Apartamento,"78,68 m²",-23.429959,-51.923021,3.0,1,2,2025-07-30,NaN
2483,https://www.silvioiwata.com.br/imovel/36200063...,Silvio Iwata,Gleba Aurora,"R$ 690.000,00",NaN,Iguaraçu,PR,NaN,Terreno em Condomínio,"637,50 m²",-23.229578,-51.861762,NaN,NaN,NaN,2025-07-30,NaN
2484,https://www.silvioiwata.com.br/imovel/36200064...,Silvio Iwata,Zona 04,"R$ 1.050.000,00",NaN,Maringá,PR,NaN,Sala Comercial em Edifício,"106,31 m²",-23.422106,-51.948861,1,4,2,2025-07-30,NaN
2485,https://www.silvioiwata.com.br/imovel/36200064...,Silvio Iwata,Zona 07,"R$ 1.100.000,00*",NaN,Maringá,PR,NaN,Apartamento,"100,91 m²",-23.414250,-51.930034,3.0,2,2,2025-07-30,NaN


In [18]:
df['district'] = df['district'].str.title()
df['price'] = pd.to_numeric(df['price'].str.replace('R$ ', '').str.replace('.', '').str.replace(',', '.'), errors='coerce')
df['type'] = df['type'].str.title()
df['city'] = df['city'].str.title().str.replace('Maringa', 'Maringá')
df['state'] = df['state'].str.upper()
df['ref'] =  df['ref'].astype(str).str.strip()
df['category'] = df['category'].str.title()
df['area'] = pd.to_numeric(df['area'].str.replace('m²', '').str.replace('.', '').str.replace(',', '.').str.strip(), errors='coerce')
df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
df['long'] = pd.to_numeric(df['long'], errors='coerce')
df['num_bedroom'] = pd.to_numeric(df['num_bedroom'], errors='coerce')
df['num_bathroom'] = pd.to_numeric(df['num_bathroom'], errors='coerce')
df['num_garage'] = pd.to_numeric(df['num_garage'], errors='coerce')
df['added_date'] = pd.to_datetime(df['added_date'], errors='coerce').dt.date
df['sold_date'] = pd.to_datetime(df['sold_date'], errors='coerce').dt.date

In [19]:
map = {
    'Apartamento': 'Apartamento',
    'Apto': 'Apartamento',
    'Apartamento Kitnet': 'Apartamento',
    'Apartamento Duplex': 'Apartamento',
    'Casa': 'Casa',
    'Casa/Sobrado': 'Casa',
    'Casa Em Condomínio': 'Casa',
    'Sobrado': 'Casa',
    'Sobrado Em Condomínio': 'Casa',
    'Prédio': 'Apartamento',
    'Predio': 'Apartamento',
    'Comercial': 'Comercial',
    'Sala Comercial': 'Comercial',
    'Sala Comercial Em Edifício': 'Comercial',
    'Salão Comercial': 'Comercial',
    'Salão': 'Comercial',
    'Barracão': 'Galpão',
    'Galpão': 'Galpão',
    'Terreno': 'Terreno',
    'Terreno Em Condomínio': 'Terreno',
    'Terreno Comercial': 'Terreno',
    'Chácara': 'Fazenda',
    'Chácara Em Área De Lazer': 'Fazenda',
    'Fazenda': 'Fazenda',
    'Sítio': 'Fazenda',
    'Sítios': 'Fazenda',
    'Rural': 'Fazenda',
    'Área': 'Casa',
    'Negócio': 'Comercial',
    'Imóvel A Beira De Rio E Lago': "Fazenda",
    'Sala': 'Comercial'
}

df['category'] = df['category'].replace(map)

In [20]:
df.loc[~np.isnan(df['sold_date']), 'Sold'] = True
df.loc[np.isnan(df['sold_date']), 'Sold'] = False

In [21]:
df

,property_url,broker,district,price,type,city,state,ref,category,area,lat,long,num_bedroom,num_bathroom,num_garage,added_date,sold_date,Sold
0,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Zona 08,NaN,Padrão,Maringá,PR,5363.0,Apartamento,55.34,-23.446941,-51.919694,2.0,1.0,NaN,2025-04-15,NaT,False
1,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Zona 01,90000.0,Padrão,Maringá,PR,31223.0,Apartamento,48.40,-23.421000,-51.933056,1.0,1.0,NaN,2025-04-15,NaT,False
2,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Zona 01,126312.1,Padrão,Maringá,PR,35423.0,Apartamento,66.36,-23.421000,-51.933056,1.0,1.0,NaN,2025-04-15,NaT,False
3,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Jardim Barcelona,135000.0,Padrão,Maringá,PR,82763.0,Terreno,300.00,-23.459239,-51.981950,NaN,NaN,NaN,2025-04-15,NaT,False
4,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Jardim Novo Paulista,140000.0,Padrão,Maringá,PR,48803.0,Terreno,200.00,-23.408781,-51.865269,NaN,NaN,NaN,2025-04-15,NaT,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482,https://www.silvioiwata.com.br/imovel/36200064...,Silvio Iwata,Zona 03,570000.0,NaN,Maringá,PR,nan,Apartamento,78.68,-23.429959,-51.923021,3.0,1.0,2.0,2025-07-30,NaT,False
2483,https://www.silvioiwata.com.br/imovel/36200063...,Silvio Iwata,Gleba Aurora,690000.0,NaN,Iguaraçu,PR,nan,Terreno,637.50,-23.229578,-51.861762,NaN,NaN,NaN,2025-07-30,NaT,False
2484,https://www.silvioiwata.com.br/imovel/36200064...,Silvio Iwata,Zona 04,1050000.0,NaN,Maringá,PR,nan,Comercial,106.31,-23.422106,-51.948861,1.0,4.0,2.0,2025-07-30,NaT,False
2485,https://www.silvioiwata.com.br/imovel/36200064...,Silvio Iwata,Zona 07,NaN,NaN,Maringá,PR,nan,Apartamento,100.91,-23.414250,-51.930034,3.0,2.0,2.0,2025-07-30,NaT,False


In [22]:
df.to_csv('silver/Properties_silver.csv', index=False, sep=';')